# Property Calculations (MDTraj)

In [1]:
# Cusotm imports
from polymer_utils import analysis, filetree, general, plotutils
from polymer_utils.representation import PolymerDir, PolymerDirManager

# General imports
from pathlib import Path
from datetime import datetime

# Numeric processing and plotting
import mdtraj as mdt
import numpy as np

# Units
from openmm.unit import nanometer, dimensionless

## Load trajectory and calculate/plot properties

In [12]:
POLY_PATH = Path('Polymers')
SAMPLE_INTERVAL = 10

mgr = PolymerDirManager(POLY_PATH)
pdir = mgr.mol_dirs['polyvinylchloride_solv_water']
dated_sims = sorted(pdir.completed_sims, key = lambda sim_dir : datetime.strptime(sim_dir.stem, general.DATETIME_FMT))
# sim_dir_to_use = dated_sims[0] # oldest 
# sim_dir_to_use = dated_sims[-1] # most recent
sim_dir_to_use = dated_sims[1] # oldest 

dated_sims

[PosixPath('Polymers/polyvinylchloride/polyvinylchloride_solv_water/MD/03-17-2023_at_10-08-46_AM'),
 PosixPath('Polymers/polyvinylchloride/polyvinylchloride_solv_water/MD/03-17-2023_at_10-42-08_AM'),
 PosixPath('Polymers/polyvinylchloride/polyvinylchloride_solv_water/MD/03-23-2023_at_12-29-39_PM')]

In [13]:
topo_path = pdir.info.structure_file
traj_path = next(sim_dir_to_use.glob('*.pdb'))

traj = mdt.load(traj_path, top=topo_path, stride=SAMPLE_INTERVAL)
traj = traj.remove_solvent(inplace=True) # don't generate new copy when de-solvating
topo = traj.topology

In [14]:
props = [
    analysis.PolyProp(calc=mdt.compute_rg                , name='Radius of Gyration'             , abbr='Rg'  , unit=nanometer),
    analysis.PolyProp(calc=mdt.shrake_rupley             , name='Solvent Accessible Surface Area', abbr='SASA', unit=nanometer**2, kwargs={'mode' : 'residue'}),
    analysis.PolyProp(calc=mdt.relative_shape_antisotropy, name='Relative Shape Anisotropy'      , abbr='K2'  , unit=dimensionless)    
]

analysis.plot_rdfs(traj, save_path=sim_dir_to_use/f'RDFs.png', header=f'Pairwise Radial Distribution Functions - {pdir.mol_name}')
analysis.plot_poly_props(traj, props, sample_interval=SAMPLE_INTERVAL, header=f'Polymer Shape and Gyration Properties - {pdir.mol_name}', save_path=sim_dir_to_use/f'shape_properties.png')
filetree.startfile(sim_dir_to_use)

<Popen: returncode: None args: ['xdg-open', PosixPath('Polymers/polyvinylchl...>

In [11]:
sim_dir_to_use

PosixPath('Polymers/polyvinylchloride/polyvinylchloride_solv_water/MD/03-23-2023_at_12-29-39_PM')

In [ ]:
mgr.all_completed_sims.keys()

## Custom Implementations of Polymer Property Calculations (MDA)

In [ ]:
import MDAnalysis as mda
from MDAnalysis import Universe
from MDAnalysis.topology.LAMMPSParser import DATAParser
from MDAnalysis.coordinates.PDB import PDBReader, PDBWriter
from MDAnalysis.topology.TOPParser import TOPParser
# from MDAnalysis.topology.PDBParser import PDBParser
# from MDAnalysis.topology.HoomdXMLParser import HoomdXMLParser

In [ ]:
coords = np.array([atom.position for atom in u.atoms])
plotutils.scatter_3D(coords)

Rg1 = u.atoms.radius_of_gyration()
G =  analysis.compute_gyration_tensor(coords)
Rg2, K2 = analysis.compute_Rg_and_K2(G)

print(Rg1, '\n', Rg2, K2)

## Some other stuff